In [1]:
# ==========================================
# 1: INSTALLATION & SETUP
# ==========================================
!pip install -qU langchain==0.1.0 langchain-google-genai==0.0.9 langchain-community==0.0.19 python-dotenv

In [2]:
pip install google-search-results

  Preparing metadata (setup.py) ... done
  Created wheel for google-search-results: filename=google_search_results-2.4.2-py3-none-any.whl size=32010 sha256=7273d8d3e8e7b2f16059299307a47df0f21f6c330aee48cbbe2e9a3b6b6cf1ff
  Stored in directory: /root/.cache/pip/wheels/0c/47/f5/89b7e770ab2996baf8c910e7353d6391e373075a0ac213519e
Successfully built google-search-results


In [3]:
import os
from google.colab import userdata
from langchain.agents import initialize_agent, AgentType, Tool
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_community.utilities import SerpAPIWrapper
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

In [4]:
# ==========================================
# 2. CONFIGURATION
#🔑 SETUP API KEY
# ==========================================
os.environ["GOOGLE_API_KEY"] = "Your_API_KEY"
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0.1,convert_system_message_to_human=True)

os.environ["SERPAPI_API_KEY"] = "Your_API_KEY"

print("✅ Libraries installed and LLM ready.")

✅ Libraries installed and LLM ready.


In [9]:
# ==========================================
# 3. DEFINE THE TOOLS (THE WORKERS)
# ==========================================
def news_search(query):

    """Worker 1: The News Reporter.
    Strictly fails if asked for numbers/prices."""

    # --- LOGIC: The "Self-Realization" of Failure ---
    # If the query is about math/money, this tool intentionally crashes
    # to trigger the hand-off.
    forbidden_keywords = ["price", "trading", "ratio", "value", "number"]
    if any(word in query.lower() for word in forbidden_keywords):
        raise ValueError("I am a News Reporter! I cannot handle specific numbers or stock prices.")

    # Otherwise, perform a news search
    search = SerpAPIWrapper(params={"tbm": "nws"}) # tbm=nws forces Google News results
    return search.run(f"{query} latest headlines")

def stock_search(query):
    """Worker 2: The Stock Broker.
    Finds specific market data."""
    #search = SerpAPIWrapper(params={"engine": "google_finance"})
    # Or standard search if finance engine not available in your plan:
    search = SerpAPIWrapper(params={"engine": "google"})
    #return search.run(f"{query} stock price") --->This code is also running

    #Following logic is to solve Identity Crisis problem -
    #This forces the LLM to trust the price even if the name looks different.
    #(e.g., Google $\to$ Alphabet, Facebook $\to$ Meta, Zomato $\to$ Eternal).

    result = search.run(f"{query} stock price")

    # <--- THE UNIVERSAL FIX --->
    # We append a GENERIC instruction to the result.
    # This tells the Agent: "If you see a price, trust it, even if the name is weird."
    return (
        f"Search Result: {result}\n"
        f"SYSTEM NOTE: The company name in the search result might differ from the query "
        f"(e.g., Parent Company Name or Legal Entity Name). "
        f"If a stock price is visible in the result, assume it is the correct data for '{query}' and return it."
    )

# Wrap them as langchain tools!
news_tool = Tool(
    name="NewsSearch",
    func=news_search,
    #description="Useful ONLY for finding text articles and headlines. Fails on numbers/prices."
    description="Useful for finding latest news and headlines about companies."
)

stock_tool = Tool(
    name="StockSearch",
    func=stock_search,
    description="Useful for finding current stock prices, market ratios, and numeric data."
)

In [10]:
# ==========================================
# 4. SHARED AGENTS
# ==========================================

# Create Individual Agents ---
# We use ZERO_SHOT_REACT_DESCRIPTION because it handles simple string inputs better.

reporter_agent = initialize_agent(
    tools=[news_tool],
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    handle_parsing_errors=True # Helps agent recover if it messes up formatting
)

stock_broker_agent = initialize_agent(
    tools=[stock_tool],
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    handle_parsing_errors=True
)

In [11]:
# ==========================================
# STRATEGY A: DECENTRALIZED (Hand-off)
# ==========================================
def run_decentralized_mode(query):
    print(f"\n🚀 MODE A: DECENTRALIZED (The Hand-off)")
    print(f"🔹 Query: '{query}'")
    print("👉 System Default: Sending to [News Reporter] first...")

    try:
        # 1. Try Reporter
        return reporter_agent.run(query)

    except ValueError as e:
        # 2. Catch Failure -> Hand off
        print(f"\n❌ News Agent REFUSED: {e}")
        print("🔄 HAND-OFF: Calling [Stock Broker]...\n")
        return stock_broker_agent.run(query)
    except Exception as e:
        print(f"\n❌ News Agent CRASHED: {e}")
        print("🔄 HAND-OFF: Calling [Stock Broker]...\n")
        return stock_broker_agent.run(query)

# ==========================================
# STRATEGY B: CENTRALIZED (The Manager)
# ==========================================

# 1. Define the Supervisor Brain
router_template = """
You are a Supervisor. Classify the user's intent.
Query: {query}
If the user wants text news/headlines: return "NEWS"
If the user wants numbers/prices/data: return "STOCK"
Category:"""
router_prompt = PromptTemplate(template=router_template, input_variables=["query"])
router_chain = LLMChain(llm=llm, prompt=router_prompt)

def run_centralized_mode(query):
    print(f"\n🚀 MODE B: CENTRALIZED (The Manager)")
    print(f"🔹 Query: '{query}'")

    # 1. The Manager decides FIRST
    print("🧠 Supervisor: analyzing intent...")
    category = router_chain.run(query).strip().upper()

    if "STOCK" in category:
        print("👉 Supervisor: Route to [Stock Broker].")
        return stock_broker_agent.run(query)
    else:
        print("👉 Supervisor: Route to [News Reporter].")
        return reporter_agent.run(query)


In [12]:
# ==========================================
# MAIN EXECUTION MENU
# ==========================================
if __name__ == "__main__":
    # Test Data
    price_question = "What is Zomato trading at?"

    # --- DEMO 1: DECENTRALIZED ---
    # Result: News agent crashes, Broker saves it.
    print("--------------------------------------------------")
    answer_a = run_decentralized_mode(price_question)
    print(f"📢 Final Answer (Mode A): {answer_a}")

    # --- DEMO 2: CENTRALIZED ---
    # Result: Supervisor sends it straight to Broker (No crash, faster).
    print("--------------------------------------------------")
    answer_b = run_centralized_mode(price_question)
    print(f"📢 Final Answer (Mode B): {answer_b}")
    print("--------------------------------------------------")

--------------------------------------------------

🚀 MODE A: DECENTRALIZED (The Hand-off)
🔹 Query: 'What is Zomato trading at?'
👉 System Default: Sending to [News Reporter] first...


> Entering new AgentExecutor chain...
Action: NewsSearch
Action Input: Zomato stock price
❌ News Agent REFUSED: I am a News Reporter! I cannot handle specific numbers or stock prices.
🔄 HAND-OFF: Calling [Stock Broker]...



> Entering new AgentExecutor chain...
Action: StockSearch
Action Input: Zomato
Observation: Search Result: {'type': 'finance_results', 'title': 'News', 'exchange': 'NSE', 'stock': 'ETERNAL', 'currency': 'INR', 'price': 283.85}
SYSTEM NOTE: The company name in the search result might differ from the query (e.g., Parent Company Name or Legal Entity Name). If a stock price is visible in the result, assume it is the correct data for 'Zomato' and return it.
Thought:I now know the final answer
Final Answer: Zomato is trading at 283.85 INR.

> Finished chain.
📢 Final Answer (Mode A): Zomato